# Predicting Adult Income via Demographics Data
by Chun-Mien (Alan) Liu, Rebecca Rosette Nanfuka, Roganci Fontelera, Yonas Gebre Marie


## Summary
Here, we tried to find the classification models with the highest accuracy on predicting if an individual's income is greater than 50K/yr based on census data. Our final classifier has a reasonable performance on the unseen test data. We observed the Logistic Regression test accuracy of $0.848$ and the RBF-SVM test accuracy of $0.855$.

The prediction model can possibly be further improved by pruning more features that are irrelevant to the prediction, since we are using almost all of the features for fitting the classifiers.


## Introduction
**TODO** by Rebecca


## Methods
### Data
The data set used here is from UC Irvine Machine Learning Repository, extracted by Barry Becker from the 1994 Census database [1].

### Analysis
The algorithms below are considered for predicting whether if an individual's income was above 50K/year or not: decision tree, k-nearest neighbors (k-nn), support vector machine with RBF kernel (SVM-RBF), logistic regression, gaussian naive bayes.
All variables were used to fit the model, with the exception of:
- `fnlwgt`: useless, since each row has a unique value
- `education-num`: due to redundancy with `education`
- `race`: for ethical reasons.

Data was split with 80% into the test set, and 20% into the train set to increase training time due to the vast amount of data. Imputation, one-hot encoding, and ordinal encoding are all done accordingly with details explained below. For model selection, we first conduct 5-fold cross-validation on all the models with default hyperparameters. Then, we select the top two models (Logistic Regression and SVM-RBF) to conduct a random search of hyperparameters with 3-fold cross-validation using accuracy as the classification metric. Finally, we used the best estimators of each to determine their accuracies on unseen test data.


## Code Portion
### Load Data


In [1]:
# Load Data In

from ucimlrepo import fetch_ucirepo 
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

Download the Adult income data from UCI using `fetch_ucirepo(id=2)` so we start from the original columns and labels each time.

In [ ]:
# fetch dataset 
adult = fetch_ucirepo(id=2) 
 
# data (as pandas dataframes)  
X = pd.DataFrame(adult.data.features) 
y = pd.DataFrame(adult.data.targets) 
 
adult_df = pd.concat([X, y], axis = 1)

# persist combined dataset to CSV
adult_df.to_csv("../data/adult.csv", index=False)

If the CSV copy is already saved, load it from disk to avoid re-downloading and to keep the same starting point for every run.

In [89]:
# read dataset from CSV
adult_df = pd.read_csv("../data/adult.csv")
adult_df.shape

(48842, 15)

Make an early stratified train/test split (80% test here because the dataset is large) so everything after this cell only uses the training data and we avoid leakage.

In [41]:
# Clean labels and split early to avoid leakage in EDA
from sklearn.model_selection import train_test_split

# Train/test split placed at the start of analysis
data_train, data_test = train_test_split(
    adult_df,
    test_size=0.8,
    random_state=42,
    stratify=adult_df['income']
)

# Use training slice for EDA to avoid peeking at test data
adult_df = data_train.reset_index(drop=True)
data_test = data_test.reset_index(drop=True)


## Summary
Check head/tail, shape, data types, and basic stats to get a quick feel for the data and catch obvious issues before doing more work.

In [42]:
adult_df.head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,45,Private,209912,Bachelors,13,Married,Exec-managerial,Husband,Asian-Pac-Islander,Male,0,0,40,India,>50K
1,44,Private,263871,HighGrad,9,Married,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
2,48,Private,180695,CommunityCollege,10,Separated,Exec-managerial,Not-in-family,White,Female,0,0,38,United-States,<=50K
3,34,Private,188246,HighGrad,9,Married,Handlers-cleaners,Husband,White,Male,0,0,40,NaN,<=50K
4,28,State-gov,158834,Masters,14,Married,Exec-managerial,Husband,White,Male,0,0,50,United-States,>50K


In [43]:
adult_df.tail(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
3902,43,Federal-gov,47902,HighGrad,9,Married,Adm-clerical,Husband,White,Male,3908,0,40,United-States,<=50K
3903,22,Local-gov,238831,CommunityCollege,10,NotMarried,Protective-serv,Own-child,White,Male,0,0,40,United-States,<=50K
3904,51,Private,110458,Bachelors,13,Separated,Prof-specialty,Not-in-family,White,Male,0,0,40,United-States,<=50K
3905,41,Private,462964,HighGrad,9,Separated,Exec-managerial,Not-in-family,White,Male,2174,0,50,United-States,<=50K
3906,24,Private,108495,CommunityCollege,10,NotMarried,Adm-clerical,Own-child,White,Male,0,0,40,India,<=50K


In [44]:
adult_df.shape

(3907, 15)

In [45]:
adult_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3907 entries, 0 to 3906
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             3907 non-null   int64 
 1   workclass       3687 non-null   object
 2   fnlwgt          3907 non-null   int64 
 3   education       3907 non-null   object
 4   education-num   3907 non-null   int64 
 5   marital-status  3907 non-null   object
 6   occupation      3686 non-null   object
 7   relationship    3907 non-null   object
 8   race            3907 non-null   object
 9   sex             3907 non-null   object
 10  capital-gain    3907 non-null   int64 
 11  capital-loss    3907 non-null   int64 
 12  hours-per-week  3907 non-null   int64 
 13  native-country  3840 non-null   object
 14  income          3907 non-null   object
dtypes: int64(6), object(9)
memory usage: 458.0+ KB


In [46]:
adult_df.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
age,3907.0,38.96,13.66,17.0,28.0,37.0,48.0,90.0
fnlwgt,3907.0,190380.57,105686.67,19302.0,118513.5,178551.0,237231.0,1184622.0
education-num,3907.0,10.08,2.62,1.0,9.0,10.0,13.0,16.0
capital-gain,3907.0,1192.89,8005.62,0.0,0.0,0.0,0.0,99999.0
capital-loss,3907.0,92.19,412.26,0.0,0.0,0.0,0.0,3004.0
hours-per-week,3907.0,40.33,12.21,1.0,40.0,40.0,45.0,99.0


In [47]:
adult_df.nunique()

age                 67
workclass            8
fnlwgt            3650
education            6
education-num       16
marital-status       4
occupation          14
relationship         6
race                 5
sex                  2
capital-gain        77
capital-loss        46
hours-per-week      76
native-country      38
income               2
dtype: int64

In [48]:
adult_df.isnull().sum()

age                 0
workclass         220
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        221
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country     67
income              0
dtype: int64

Use summary tables (shape, types, describe, unique counts, missing values) to see how big the data is, how many categories we have, and where values are missing.

In [49]:
for col in adult_df.columns:
    print(f"Unique values in column '{col}':")
    print(adult_df[col].unique())
    print("-" * 20)

Unique values in column 'age':
[45 44 48 34 28 27 30 29 33 25 36 47 32 24 23 19 41 66 56 17 31 38 63 52
 22 49 42 35 26 21 51 40 46 67 39 18 62 59 37 43 68 54 74 20 50 53 60 64
 57 65 55 72 71 75 58 73 61 69 84 70 77 79 81 80 78 90 76]
--------------------
Unique values in column 'workclass':
['Private' 'State-gov' 'Self-emp-inc' 'Local-gov' nan 'Federal-gov'
 'Self-emp-not-inc' 'Without-pay' 'Never-worked']
--------------------
Unique values in column 'fnlwgt':
[209912 263871 180695 ... 110458 462964 108495]
--------------------
Unique values in column 'education':
['Bachelors' 'HighGrad' 'CommunityCollege' 'Masters' 'dropout' 'Doctorate']
--------------------
Unique values in column 'education-num':
[13  9 10 14  4 11  6 12 15  7  3  8  2  5 16  1]
--------------------
Unique values in column 'marital-status':
['Married' 'Separated' 'NotMarried' 'Widowed']
--------------------
Unique values in column 'occupation':
['Exec-managerial' 'Craft-repair' 'Handlers-cleaners' 'Farming-fishing

## Clean up
Turn `?` into `NaN`, standardize the income labels to `<=50K` and `>50K`, and group education and marital status levels to reduce noise and make modeling easier.

In [50]:
adult_df.replace('?', np.nan, inplace=True)

In [51]:
adult_df.isnull().sum()

age                 0
workclass         220
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        221
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country     67
income              0
dtype: int64

In [52]:
adult_df['income'] = adult_df['income'].str.replace('<=50K.', '<=50K')
adult_df['income'] = adult_df['income'].str.replace('>50K.', '>50K')

In [53]:
education_mapping = {
    'Preschool': 'dropout',
    '10th': 'dropout',
    '11th': 'dropout',
    '12th': 'dropout',
    '1st-4th': 'dropout',
    '5th-6th': 'dropout',
    '7th-8th': 'dropout',
    '9th': 'dropout',
    'HS-Grad': 'HighGrad',
    'HS-grad': 'HighGrad',
    'Some-college': 'CommunityCollege',
    'Assoc-acdm': 'CommunityCollege',
    'Assoc-voc': 'CommunityCollege',
    'Masters': 'Masters',
    'Prof-school': 'Masters',
}

adult_df['education'] = adult_df['education'].replace(education_mapping)

In [54]:
marital_status_mapping = {
    'Never-married': 'NotMarried',
    'Married-AF-spouse': 'Married',
    'Married-civ-spouse': 'Married',
    'Married-spouse-absent': 'NotMarried',
    'Separated': 'Separated',
    'Divorced': 'Separated',
    'Widowed': 'Widowed'
}

adult_df['marital-status'] = adult_df['marital-status'].replace(marital_status_mapping)

# EDA
Plot key distributions for age, education, workclass, race, marital status, and native country. Also look at the income class balance to guide feature choices and any class weighting.

In [55]:
alt.Chart(adult_df).mark_bar().encode(
    alt.X('age:Q', bin=alt.Bin(maxbins=20), title='Age'), 
    alt.Y('count():Q', title='Count'), 
    color='income:N', 
).properties(
    width=600, 
    height=400 
)

alt.Chart(...)

In [56]:
alt.Chart(adult_df).transform_density(
     'age',
     groupby=['income'],
     as_=['age', 'density'],
).mark_area(
     opacity=0.4
 ).encode(
     x='age',
     y=alt.Y('density:Q').stack(False),
     color='income'
)

alt.Chart(...)

In [57]:
alt.Chart(adult_df).mark_bar().encode(
    alt.X('count():Q', title='Count'),
    alt.Y('marital-status:N', title='Marital Status').sort('x')
).properties(
    width=300,  
    height=100 
)

alt.Chart(...)

In [58]:
alt.Chart(adult_df).mark_bar().encode(
    alt.X('count():Q', title='Count'),
    alt.Y('race:N', title='Race').sort('x')
).properties(
    width=300,  
    height=200 
)

alt.Chart(...)

In [59]:
alt.Chart(adult_df).mark_bar().encode(
    x='count()',
    y=alt.Y('education', title='Education').sort('x')
).properties(
    width=300,  
    height=200 
)

alt.Chart(...)

In [60]:
alt.Chart(adult_df).mark_bar().encode(
    x='count()',
    y=alt.Y('workclass', title='Work Class').sort('x')
).properties(
    width=300,  
    height=200 
)

alt.Chart(...)

In [61]:
adult_df['native-country'].value_counts()

native-country
United-States         3506
Mexico                  88
Germany                 17
Canada                  16
El-Salvador             16
Cuba                    15
Philippines             13
Puerto-Rico             13
South                   12
England                 11
Dominican-Republic      10
China                   10
Vietnam                  9
Nicaragua                8
Jamaica                  7
Japan                    7
Iran                     7
India                    7
Guatemala                6
Columbia                 6
Italy                    5
Hong                     5
Peru                     5
Poland                   5
Portugal                 4
France                   4
Ecuador                  4
Haiti                    4
Ireland                  4
Greece                   3
Honduras                 2
Thailand                 2
Laos                     2
Taiwan                   2
Scotland                 2
Cambodia                 1
Trinadad&Toba

In [62]:
# alt.Chart(adult_df).mark_point(opacity=0.6, size=2).encode(
#     alt.X(alt.repeat('row')).type('quantitative'),
#     alt.Y(alt.repeat('column')).type('quantitative'),
#     color='income'  # Map 'prediction' to color (nominal type)
# ).properties(
#     width=130,
#     height= 130
# ).repeat(
#     column=['age','education-num','capital-gain','capital-loss','hours-per-week'],
#     row=['age', 'education-num','capital-gain','capital-loss','hours-per-week']
# )

In [63]:
# Altair correlation heatmap for numeric features
corr = adult_df.corr(numeric_only=True)
numeric_cols = corr.columns.tolist()

corr_long = corr.stack().reset_index()
corr_long.columns = ['var1', 'var2', 'corr']

heatmap = alt.Chart(corr_long).mark_rect().encode(
    x=alt.X('var1:N', sort=numeric_cols, title=''),
    y=alt.Y('var2:N', sort=list(reversed(numeric_cols)), title=''),
    color=alt.Color('corr:Q', scale=alt.Scale(scheme='redblue', domain=[-1, 1]), title='Correlation'),
    tooltip=[alt.Tooltip('var1:N', title='Feature 1'),
             alt.Tooltip('var2:N', title='Feature 2'),
             alt.Tooltip('corr:Q', title='Correlation', format='.2f')]
).properties(width=480, height=480)

labels = alt.Chart(corr_long).mark_text(baseline='middle', color='black', size=12).encode(
    x=alt.X('var1:N', sort=numeric_cols),
    y=alt.Y('var2:N', sort=list(reversed(numeric_cols))),
    text=alt.Text('corr:Q', format='.2f')
)

(heatmap + labels).configure_axis(labelAngle=-45)

alt.LayerChart(...)

In [64]:
alt.Chart(adult_df).mark_bar().encode(
    alt.X('income:N', title='Income'),
    alt.Y('count():Q', title='Count'), 
    color='income:N' 
).properties(
    width=400, 
    height=300  
)

alt.Chart(...)

**Figure 1.** Income class distribution (Altair bar chart). Counts for `<=50K` vs `>50K` show the class mix before modeling.

## Preprocessing pipeline
Set up a pipeline that imputes missing values, encodes categoricals (with ordered education), scales numerics, and applies the same steps to both train and test data.

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Separate features and target using the pre-created train/test split
numeric_features = ['age', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'native-country']
ordinal_features = ['education']
binary_features = ['sex']
drop_features = ['fnlwgt', 'education-num', 'race']
target = "income"

# Ordered levels for the education column (lowest to highest)
education_order = [
    'dropout',
    'HighGrad',
    'CommunityCollege',
    'Bachelors',
    'Masters',
    'Doctorate',
]

X_train = adult_df.drop(columns=drop_features + [target])
X_test = data_test.drop(columns=drop_features + [target])
y_train = adult_df[target]
y_test = data_test[target]


## Column preprocessing
List which columns are numeric, categorical, ordinal, binary, and which to drop. Set the target column, then build pipelines for each group before combining them.

In [66]:
# Pipelines for different data types
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

ordinal_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(categories=[education_order], handle_unknown='use_encoded_value', unknown_value=-1))
])

binary_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse_output=False))
])

numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('ordinal', ordinal_pipeline, ordinal_features),
        ('binary', binary_pipeline, binary_features),
        ('categorical', categorical_pipeline, categorical_features),
        ('numeric', numeric_pipeline, numeric_features),
    ]
)

preprocessor


,transformers,"[('ordinal', ...), ('binary', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,missing_values,nan
,strategy,'most_frequent'
,fill_value,None


**Preprocessing design.** Numerics: median impute, then scale. Categoricals: most-frequent impute, then one-hot. Education: ordered encoder. Sex: binary one-hot. A `ColumnTransformer` applies the same steps to train and test.

In [67]:
# generate preprocessed training data just to check how it looks
X_train_processed = preprocessor.fit_transform(X_train)

# get the feature names after preprocessing
def get_feature_names(preprocessor):
    feature_names = []

    # Ordinal features
    ordinal_features = preprocessor.transformers_[0][2]
    feature_names.extend(ordinal_features)

    # Binary features
    binary_features = preprocessor.transformers_[1][2]
    binary_encoder = preprocessor.transformers_[1][1].named_steps['encoder']
    binary_feature_names = binary_encoder.get_feature_names_out(binary_features)
    feature_names.extend(binary_feature_names)

    # Categorical features
    categorical_features = preprocessor.transformers_[2][2]
    categorical_encoder = preprocessor.transformers_[2][1].named_steps['encoder']
    categorical_feature_names = categorical_encoder.get_feature_names_out(categorical_features)
    feature_names.extend(categorical_feature_names)

    # Numeric features
    numeric_features = preprocessor.transformers_[3][2]
    feature_names.extend(numeric_features)

    return feature_names

feature_names = get_feature_names(preprocessor)

# convert processed data back to DataFrame for better readability
X_train_processed_df = pd.DataFrame(X_train_processed, columns=feature_names)
X_train_processed_df.head()

,education,sex_Male,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,...,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,age,capital-gain,capital-loss,hours-per-week
0,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.442511,-0.149026,-0.223656,-0.027201
1,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.369284,-0.149026,-0.223656,-0.027201
2,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.662192,-0.149026,-0.223656,-0.191077
3,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,-0.362986,-0.149026,-0.223656,-0.027201
4,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,-0.802349,-0.149026,-0.223656,0.792181


### Baseline and model comparison
Start with a dummy classifier, then compare tree, kNN, SVM, GaussianNB, and Logistic Regression using the shared preprocessing to see which models look promising.

In [90]:
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate


In [91]:
# Code adapted from MDS 571 course materials
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores.iloc[i], std_scores.iloc[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [70]:
models = {
    'Dummy-most_frequent': Pipeline([
        ('preprocess', preprocessor),
        ('classifier', DummyClassifier(strategy='most_frequent'))
    ]),
    'DecisionTree': Pipeline([
        ('preprocess', preprocessor),
        ('classifier', DecisionTreeClassifier(random_state=42))
    ]),
    'KNN': Pipeline([
        ('preprocess', preprocessor),
        ('classifier', KNeighborsClassifier())
    ]),
    'SVM-RBF': Pipeline([
        ('preprocess', preprocessor),
        ('classifier', SVC(kernel='rbf'))
    ]),
    'LogisticRegression': Pipeline([
        ('preprocess', preprocessor),
        ('classifier', LogisticRegression(max_iter=200))
    ]),
    'GaussianNB': Pipeline([
        ('preprocess', preprocessor),
        ('classifier', GaussianNB())
    ]),
}

cv_results = {}
for name, model in models.items():
    cv_results[name] = mean_std_cross_val_scores(
        model, X_train, y_train, cv=5, return_train_score=True
    )

cv_summary = pd.DataFrame(cv_results).T
cv_summary


,fit_time,score_time,test_score,train_score
Dummy-most_frequent,0.010 (+/- 0.003),0.004 (+/- 0.001),0.761 (+/- 0.000),0.761 (+/- 0.000)
DecisionTree,0.015 (+/- 0.001),0.003 (+/- 0.000),0.814 (+/- 0.014),0.991 (+/- 0.001)
KNN,0.009 (+/- 0.001),0.008 (+/- 0.001),0.826 (+/- 0.014),0.875 (+/- 0.002)
SVM-RBF,0.101 (+/- 0.004),0.053 (+/- 0.001),0.856 (+/- 0.009),0.873 (+/- 0.002)
LogisticRegression,0.023 (+/- 0.002),0.003 (+/- 0.000),0.850 (+/- 0.003),0.856 (+/- 0.002)
GaussianNB,0.010 (+/- 0.000),0.003 (+/- 0.000),0.350 (+/- 0.045),0.356 (+/- 0.032)


The best perfoming models are Logistic Regression and RBF-SVM, so we tune hyperparameters for those next.

### Hyperparameter tuning for top models
Run randomized search for Logistic Regression (C) and RBF-SVM (C, gamma) with the same preprocessing. Use cross-validated accuracy to pick the settings.

In [71]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform


In [73]:
log_reg_pipe = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('classifier', LogisticRegression(max_iter=500, random_state=42))
])

log_reg_params = {
    'classifier__C': loguniform(1e-3, 1e3),
    'classifier__penalty': ['l2'],
    'classifier__solver': ['lbfgs'],
}

log_reg_search = RandomizedSearchCV(
    log_reg_pipe,
    param_distributions=log_reg_params,
    n_iter=200,
    cv=3,
    scoring='accuracy',
    random_state=42,
    n_jobs=1,
)

log_reg_search.fit(X_train, y_train)

print(f"Best Logistic Regression CV accuracy: {log_reg_search.best_score_:.3f}")
print("Best params:", log_reg_search.best_params_)


Best Logistic Regression CV accuracy: 0.850
Best params: {'classifier__C': np.float64(1.7220805097801954), 'classifier__penalty': 'l2', 'classifier__solver': 'lbfgs'}


Tuning only moved accuracy a little, so expect small gains and keep track of the chosen parameters.

In [75]:
svm_pipe = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('classifier', SVC(kernel='rbf', random_state=42))
])

svm_params = {
    'classifier__C': loguniform(1e-2, 1e2),
    'classifier__gamma': loguniform(1e-4, 1e0),
}

svm_search = RandomizedSearchCV(
    svm_pipe,
    param_distributions=svm_params,
    n_iter=20,
    cv=3,
    scoring='accuracy',
    random_state=42,
    n_jobs=1,
)

svm_search.fit(X_train, y_train)

print(f"Best SVM (RBF) CV accuracy: {svm_search.best_score_:.3f}")
print("Best params:", svm_search.best_params_)


Best SVM (RBF) CV accuracy: 0.856
Best params: {'classifier__C': np.float64(2.5378155082656657), 'classifier__gamma': np.float64(0.06796578090758151)}


### Test set evaluation
Score the tuned Logistic Regression and RBF-SVM on the held-out test set to report how well they generalize.

In [80]:
# Use tuned models when available; otherwise fall back to the quick baseline fits
best_log_reg = log_reg_search.best_estimator_
log_reg_name = "Tuned Logistic Regression"

best_svm = svm_search.best_estimator_
svm_name = "Tuned RBF-SVM"


for name, model in [
    (log_reg_name, best_log_reg),
    (svm_name, best_svm),
]:
    test_acc = model.score(X_test, y_test)
    print(f"{name} test accuracy: {test_acc:.3f}")


Tuned Logistic Regression test accuracy: 0.848
Tuned RBF-SVM test accuracy: 0.855


### Model comparison and interpretation
RBF-SVM is a bit more accurate but harder to explain. Logistic Regression is close in accuracy and its coefficients and odds ratios make feature effects clear. Use LR when you need explanations; use SVM if you want the tiny accuracy edge.

## Logistic Regression interpretation

Lets use Logistic Regression coefficients to rank features.

In [92]:
import numpy as np
import pandas as pd

log_reg_clf = best_log_reg.named_steps['classifier']
feature_names = best_log_reg.named_steps['preprocess'].get_feature_names_out()
coefs = log_reg_clf.coef_[0]

coef_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefs,
    'odds_ratio': np.exp(coefs)
})

# Strongest positive and negative signals
top_positive = coef_df.sort_values('coefficient', ascending=False).head(10)
top_negative = coef_df.sort_values('coefficient').head(10)

print("Top positive predictors (increase odds of >50K):")
display(top_positive)

print("Top negative predictors (decrease odds of >50K):")
display(top_negative)


Top positive predictors (increase odds of >50K):


,feature,coefficient,odds_ratio
73,numeric__capital-gain,2.496440,12.139203
33,categorical__relationship_Wife,1.473345,4.363810
10,categorical__marital-status_Married,1.179246,3.251921
26,categorical__occupation_Tech-support,1.109182,3.031877
43,categorical__native-country_France,1.094701,2.988290
42,categorical__native-country_England,1.060307,2.887256
50,categorical__native-country_Hungary,1.015530,2.760826
67,categorical__native-country_Taiwan,0.885851,2.425047
17,categorical__occupation_Exec-managerial,0.866976,2.379705
49,categorical__native-country_Hong,0.833562,2.301503


Top negative predictors (decrease odds of >50K):


,feature,coefficient,odds_ratio
21,categorical__occupation_Other-service,-1.434383,0.238262
30,categorical__relationship_Other-relative,-1.233262,0.291341
22,categorical__occupation_Priv-house-serv,-1.146609,0.317712
45,categorical__native-country_Greece,-1.075132,0.341253
18,categorical__occupation_Farming-fishing,-0.932763,0.393465
65,categorical__native-country_Scotland,-0.923698,0.397048
31,categorical__relationship_Own-child,-0.906653,0.403874
57,categorical__native-country_Laos,-0.744338,0.475048
37,categorical__native-country_Columbia,-0.693283,0.499932
52,categorical__native-country_Iran,-0.682821,0.505190


### Visual checks for interpretability
Altair bars show the strongest positive and negative LR coefficients (Figure 2). Binned probability curves show average P(`>50K`) across deciles for `hours-per-week`, `capital-gain`, and `age` with 10th/90th percentile bands (Figure 3).

In [87]:
import altair as alt
import pandas as pd

# Altair bars for top coefficients
pos_chart = (
    alt.Chart(top_positive)
    .mark_bar(color='seagreen')
    .encode(
        y=alt.Y('feature:N', sort=list(top_positive['feature'][::-1]), title='Feature'),
        x=alt.X('coefficient:Q', title='Coefficient (log-odds)'),
        tooltip=['feature', 'coefficient', 'odds_ratio']
    )
    .properties(title='Top positive coefficients', width=280, height=230)
)

neg_chart = (
    alt.Chart(top_negative)
    .mark_bar(color='firebrick')
    .encode(
        y=alt.Y('feature:N', sort=list(top_negative['feature'][::-1]), title='Feature'),
        x=alt.X('coefficient:Q', title='Coefficient (log-odds)'),
        tooltip=['feature', 'coefficient', 'odds_ratio']
    )
    .properties(title='Top negative coefficients', width=280, height=230)
)

(pos_chart | neg_chart)


alt.HConcatChart(...)

### Results and Discussion
From the analyses above, we discovered that logistic regression and SVM RBF are the best classification models for predicting income, among all the other classification algorithms. The Logistic Regression test achieves test accuracy of $0.848$ and the RBF-SVM achieves test accuracy of $0.855$. This is somewhat surprising because logistic regression is a baseline model for classifications.

From the logistic regression coefficients, we find that key features include capital gains, occupations, native countries, and relationships, are especially related to determining income levels.

For our next step, we should potentially prune some other irrelevant features to reduce noise in the models and achieve a better result. Other ideas include adding calibration curves, simple fairness checks, and considering grouping rare country categories to make the model steadier and easier to explain.

## References
[1] Becker B, Kohavi R. Adult [dataset]. 1996. UCI Machine Learning Repository. Available from: https://doi.org/10.24432/C5XW20.
